In [30]:
import pandas as pd
import folium
import requests
from datetime import datetime

In [3]:
url = 'https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/642efeca-1258-4c23-8e86-d9affca26001/resource/b3268057-6b55-4e6e-b1ae-b752bce92a1c/download/Stationary%20Sign%20locations.geojson'
response = requests.get(url)

In [4]:
data = response.json()

In [5]:
df = pd.json_normalize(data, 'features')

In [6]:
df = df.drop('type', axis=1)

In [7]:
column_mapping = {}
for col in df.columns:
    if col.startswith("properties."):
        new_col = col[len("properties."):]
        column_mapping[col] = new_col
df.rename(columns=column_mapping, inplace=True)

In [18]:
df

,_id,address,dir,end_date,flash_speed,min_speed,schedule,sign_id,sign_name,speed_limit,start_date,strobe_speed,ward_no,geometry.type,geometry.coordinates
0,9588,2546 Kipling Avenue,SB,None,51,30,Weekdays from 7 AM - 9 PM,57,Kipling Ave SB,50,2019-04-11T05:00:00,56,1.0,Point,"[-79.583516, 43.745977]"
1,9589,51 Kingsview Boulevard,EB,None,31,20,Weekdays from 7 AM - 9 PM,51,Kingsview Blvd EB,30,2019-01-03T14:00:00,36,1.0,Point,"[-79.554847, 43.698055]"
2,9590,"1, Promenade Circle",None,2020-09-23T23:00:00,41,30,Weekdays from 7 AM - 9 PM,61902,D -,40,2023-01-24T00:00:00,46,NaN,Point,"[-79.450379788118, 43.80822462182]"
3,9591,Promenade Circle,None,None,31,20,Weekdays from 7 AM - 9 PM,2837,None,30,2022-02-06T00:00:00,36,NaN,Point,"[-79.450789057049, 43.805702646614]"
4,9592,7 Hullen Crescent,SB,2020-04-04T23:00:00,41,30,Weekdays from 7 AM - 9 PM,268,Humberwood Blvd SB,40,2017-09-01T09:00:00,46,1.0,Point,"[-79.619998, 43.729637]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
909,10497,"1, Promenade Circle",None,2020-04-03T14:00:00,51,30,Weekdays from 7 AM - 9 PM,2876,D -,50,2022-03-10T00:00:00,56,NaN,Point,"[-79.451798510369, 43.808401790985]"
910,10498,430 Shuter St,WB,2021-04-19T12:00:00,31,20,Weekdays from 7 AM - 9 PM,2631,SHUTER ST WB,30,2021-02-10T12:00:00,36,13.0,Point,"[-79.361051, 43.657892]"
911,10499,Opp 161 Gary Dr,WB,2022-03-01T16:00:00,41,5,Weekdays from 7 AM - 9 PM,2667,GARY DR WB,40,2021-04-13T10:00:00,46,5.0,Point,"[-79.516721, 43.711612]"
912,10500,18 Washburn Way,SB,2022-08-17T15:00:00,31,20,Weekdays from 7 AM - 9 PM,692,Washburn Way SB,30,2019-03-11T08:00:00,36,23.0,Point,"[-79.227773, 43.797108]"


In [28]:
# Create a Folium map 
map = folium.Map(location=[43.6532, -79.3832], zoom_start=4)
# Add markers for each address to the map    popup_text = "Address: {}<br>Latitude: {}<br>Longitude: {}".format(row['Address'], row['Latitude'], row['Longitude'])
for index, row in df.iterrows():
    popup_text = "_id: {}<br>address: {}<br>dir: {}<br>end_date: {}<br>flash_speed: {}<br>min_speed: {}<br>schedule: {}<br>sign_id: {}<br>sign_name: {}\
    <br>speed_limit: {}<br>start_date: {}<br>strobe_speed: {}<br>ward_no: {}<br>geometry.type: {}<br>Latitude: {}<br>Longitude: {}"\
    .format(row['_id'], row['address'], row['dir'], row['end_date'], row['flash_speed'], row['min_speed'], row['schedule'], row['sign_id'], 
    row['sign_name'], row['speed_limit'], row['start_date'], row['strobe_speed'], row['ward_no'], row['geometry.type'],
    row['geometry.coordinates'][1], row['geometry.coordinates'][0])
    popup = folium.Popup(popup_text, max_width=300)
    icon_anchor= (0, 0)
    icon_url = 'https://stinson.ca/media/catalog/product/cache/4425efb2f927b0d14b906c7707771a80/w/c/wc-1_school_area.png' 
    icon_size = (30, 30)  # Set the icon size in pixels
    icon = folium.features.CustomIcon(icon_url, icon_size, icon_anchor=icon_anchor)
    # Create a new marker with the custom icon and popup text and add it to the map
    marker = folium.Marker(location=[row['geometry.coordinates'][1], row['geometry.coordinates'][0]], popup=popup_text, icon=icon)
    marker.add_to(map)


In [29]:
map

In [126]:
print(df.iloc[0])

type                                         Feature
properties._id                                  9588
properties.address               2546 Kipling Avenue
properties.dir                                    SB
properties.end_date                             None
properties.flash_speed                            51
properties.min_speed                              30
properties.schedule        Weekdays from 7 AM - 9 PM
properties.sign_id                                57
properties.sign_name                 Kipling Ave SB 
properties.speed_limit                            50
properties.start_date            2019-04-11T05:00:00
properties.strobe_speed                           56
properties.ward_no                               1.0
geometry.type                                  Point
geometry.coordinates         [-79.583516, 43.745977]
Name: 0, dtype: object


In [130]:
print(df.iloc[:, :1])

     properties._id
0              9588
1              9589
2              9590
3              9591
4              9592
..              ...
909           10497
910           10498
911           10499
912           10500
913           10501

[914 rows x 1 columns]
